In [16]:
import pandas as pd
import numpy as np
import re
from nltk.metrics import jaccard_distance

import warnings
warnings.filterwarnings('ignore')	

In [17]:
ingredients = pd.read_csv('../Data/ingredients.csv')
products = pd.read_excel('../Data/products.xlsx')

In [18]:
# Adding new rows for most common misspellings
def create_new_rows(generic_name, synonym_values):
    # Find the ingredientID based on the generic_name
    ingredient_id = ingredients.loc[ingredients['generic_name'] == generic_name, 'ingredientID'].iloc[0]

    # Create new rows with the found ingredientID
    new_rows = {'ingredientID': [ingredient_id] * len(synonym_values),
                'generic_name': [generic_name] * len(synonym_values),
                'synonym': synonym_values}
    return new_rows

new_rows1 = create_new_rows('Alcohol Denat.', ['alchol dent', 'alcoholdenat', 'alcoholdent', 'alchol', 'alkohol'])
new_rows2 = create_new_rows('Alpha-Isomethyl Ionone', ['alpha, isomethyl ionon', 'alpha, isomethyl ionone', 'alpha isomethyl ionone', 'alpha,isomethyl ionon', 'alpha-isomethylionone'])
new_rows3 = create_new_rows('Dimethicone', ['dimthicone'])
new_rows4 = create_new_rows('Linalool', ['linalol'])

new_rows_list = [new_rows1, new_rows2, new_rows3, new_rows4]
for new_rows in new_rows_list:
    ingredients = pd.concat([ingredients, pd.DataFrame(new_rows)], ignore_index=True)

In [19]:
# Converting to list
synonyms = ingredients['synonym'].tolist()
ingredient_names = products['ingredient_list'].tolist()

In [20]:
def find_first_match(ingredient, candidates, threshold=0.50):
    ingredient_set = set(re.split(r'\W+', ingredient))

    # Set to keep track of matched ingredients
    matched_ingredients = set()

    for candidate in candidates:
        # Skip if candidate ingredient has already been matched
        if pd.notna(candidate):  # Check if candidate is not NaN
            if candidate in matched_ingredients:
                continue

            if isinstance(candidate, str):  # Check if candidate is a string
                candidate_set = set(re.split(r'\W+', candidate))
                union_size = len(ingredient_set.union(candidate_set))

                if union_size != 0:
                    distance = jaccard_distance(ingredient_set, candidate_set)
                    similarity = 1 - distance

                    if similarity > threshold:
                        # Add the matched ingredient to the set
                        matched_ingredients.add(candidate)
                        
                        # Return the first match and exit the function
                        return candidate
    return None

In [21]:

matches_df = pd.DataFrame(columns=['Matching Ingredient', 'Synonym'])

dfs = []

for ingredient in ingredient_names:
    if isinstance(ingredient, str):
        direct_match = [synonym for synonym in synonyms if synonym == ingredient] or [None]

        if direct_match[0] == None:
            match = find_first_match(ingredient, synonyms)
        else:
            match = direct_match[0]

        df = pd.DataFrame({'Matching Ingredient': [ingredient], 'Synonym': [match]})

        dfs.append(df)

matches_df = pd.concat(dfs, ignore_index=True)

In [22]:
synonym_to_generic = dict(zip(ingredients['synonym'], ingredients['generic_name']))

# Add a new column to matches_df to store the corresponding generic name
matches_df['Generic Name'] = None

# Loop through each ingredient
for index, row in matches_df.iterrows():
    ingredient = row['Matching Ingredient']
    match = row['Synonym']

    # Check if the match is not None
    if match is not None:
        # Use the dictionary to find the corresponding generic name
        generic_name = synonym_to_generic.get(match)

        # Update the 'Generic_Name' column in matches_df
        matches_df.at[index, 'Generic Name'] = generic_name

In [27]:
none_count = 0
for value in matches_df['Synonym']:
    if str(value).strip().lower() == 'none':
        none_count += 1

matched = round((len(products) - none_count)/len(products), 4)*100
not_matched = round((100-matched), 6)
exact_matches = products['ingredient_list'][products['ingredient_list'].isin(ingredients['synonym'].values)].tolist()
exact_matches = round(len(exact_matches)/len(products['ingredient_list']), 4)*100

print(f'MATCHED ingredients:               {round(matched, 4)} % \t/ {len(ingredient_names) - none_count}')
print(f'NOT MATCHED ingredients:           {not_matched} % \t/ {none_count}')
print()
print(f'EXACT MATCH of ingredients:        {exact_matches} %')
print(f'JACCARD DISTANCE MATCH of ing.:    {round(matched-exact_matches, 4)} %')

matches_df

MATCHED ingredients:               91.74 % 	/ 210283
NOT MATCHED ingredients:           8.26 % 	/ 18925

EXACT MATCH of ingredients:        82.0 %
JACCARD DISTANCE MATCH of ing.:    9.74 %


,Matching Ingredient,Synonym,Generic Name
0,aqua/water,aqua/water,Water
1,aluminum chlorohydrate,aluminum chlorohydrate,Aluminum Chlorohydrate
2,cetearyl alcohol,cetearyl alcohol,Cetearyl Alcohol
3,ceteareth-33,ceteareth-33,Ceteareth-33
4,parfum/fragrance,parfum/fragrance,Parfum
...,...,...,...
229203,charcoal powder,charcoal powder,Charcoal Powder
229204,parfum (fragrance),parfum (fragrance),Parfum
229205,limonene,limonene,Limonene
229206,linalool,linalool,Linalool


In [24]:
matches_df[['Generic Name', 'Synonym', 'Matching Ingredient']].to_excel('matches_modified_split_0.50.xlsx', index=False)

In [25]:
# Analysis of unmatched ingredients

filtered_df = matches_df[matches_df['Generic Name'].isna()]
value_counts = filtered_df['Matching Ingredient'].value_counts()

# Print the first 20 most frequent values and their frequencies
for value, frequency in value_counts.head(60).iteritems():
    print(f"Value: {value}, Frequency: {frequency}")

Value: lemonellol, Frequency: 196
Value: euphorbia cerifera (candelilla) wax, Frequency: 153
Value: titanium dioxide (nano), Frequency: 135
Value: paraffinum liquidum mineral oil, Frequency: 134
Value: lime, Frequency: 102
Value: hexxl cinnamal, Frequency: 90
Value: benzyl alchol, Frequency: 90
Value: cera microcristallina (microcrystalline wax), Frequency: 86
Value: huile minerale, Frequency: 79
Value: cetearyl alchol, Frequency: 69
Value: ceyl esters, Frequency: 68
Value: solum diatomeae/diatomaceous earth/terre de diatomees, Frequency: 65
Value: copernicia cerifera cera (copernicia cerifera (carnauba) wax), Frequency: 65
Value: cera microcristallina microcrystalline wax, Frequency: 61
Value: sodium, Frequency: 61
Value: hydroxyylcellulose, Frequency: 58
Value: ci 16035 (red 40), Frequency: 55
Value: candelilla cera (euphorbia cerifera (candelilla) wax), Frequency: 54
Value: red 33 (ci 17200), Frequency: 49
Value: ceyl alcohol, Frequency: 48
Value: yellow 6 (ci 15985), Frequency: 46


In [26]:
exact_matches = products['ingredient_list'][products['ingredient_list'].isin(ingredients['synonym'].values)].tolist()
print('Procent na exact match:            ', round(len(exact_matches)/len(products['ingredient_list']), 4)*100, '%')
print('Cela lista na sostojki vo produkti:', len(products['ingredient_list']))
print('Pogodeni:                          ', len(exact_matches))
print('Nepogodeni:                        ', len(products['ingredient_list']) - len(exact_matches))

Procent na exact match:             82.0 %
Cela lista na sostojki vo produkti: 229208
Pogodeni:                           187947
Nepogodeni:                         41261
